In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import binance_api
import time
import position_size
import sys
from datetime import date, timedelta
from binance.client import Client

In [2]:
# CGLD on CoinAPI is CELO on Binance. Make sure we map them: CoinAPI ➝ Binance
name_map = {
  'CGLD': 'CELO'
}

API_KEY = '0tjl3JlxuBgs60yfMPnT1mQAPXKboCQjEG1wpLlblQe0Ab26cbxqriRFSrSYyU76'
API_SECRET = 'V1Auz9PwxFCJH11daLRNf3BIp5zbJZjiaGueDxBsUY0zvfMfR4UIZAa9M7VvnJun'
client = Client(API_KEY, API_SECRET, testnet=True)

In [3]:
current_positions = client.get_exchange_info()
print(current_positions)
print('----')
# holdings = [
#     (bal['asset'], total)
#     for bal in current_positions['assets']
#     if (total := float(bal.get('free', 0)) + float(bal.get('locked', 0))) > 0
# ]
# # for symbol in current_positions:
# #     print(f'{symbol}')
# #     print(client.get_symbol_info(f'{symbol}USDT'))
# all_cryptos = client.get_exchange_info()
# print(all_cryptos)

# print(f'Current positions are: {current_positions}')


{'timezone': 'UTC', 'serverTime': 1760510385528, 'rateLimits': [{'rateLimitType': 'REQUEST_WEIGHT', 'interval': 'MINUTE', 'intervalNum': 1, 'limit': 6000}, {'rateLimitType': 'ORDERS', 'interval': 'SECOND', 'intervalNum': 10, 'limit': 50}, {'rateLimitType': 'ORDERS', 'interval': 'DAY', 'intervalNum': 1, 'limit': 160000}, {'rateLimitType': 'RAW_REQUESTS', 'interval': 'MINUTE', 'intervalNum': 5, 'limit': 61000}], 'exchangeFilters': [], 'symbols': [{'symbol': 'ETHBTC', 'status': 'TRADING', 'baseAsset': 'ETH', 'baseAssetPrecision': 8, 'quoteAsset': 'BTC', 'quotePrecision': 8, 'quoteAssetPrecision': 8, 'baseCommissionPrecision': 8, 'quoteCommissionPrecision': 8, 'orderTypes': ['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS', 'STOP_LOSS_LIMIT', 'TAKE_PROFIT', 'TAKE_PROFIT_LIMIT'], 'icebergAllowed': True, 'ocoAllowed': True, 'otoAllowed': True, 'quoteOrderQtyMarketAllowed': True, 'allowTrailingStop': True, 'cancelReplaceAllowed': True, 'amendAllowed': True, 'pegInstructionsAllowed': True, 'isSpo

In [ ]:
theoretical_positions = pd.read_csv('weights.csv', index_col='date', parse_dates=['date'])
renamed_positions = theoretical_positions.rename(columns=name_map)
# TODO: Fix -2!
all_future_weights = renamed_positions.iloc[-2]
future_weights = all_future_weights[all_future_weights != 0]
future_weights_date = renamed_positions.index[-1].date()

print(f'Future weights are: {future_weights.to_dict()}')

In [10]:
# Get infos about current positions
for symbol, amount in current_positions.items():
    print(f'{symbol}: {amount}')
    print(binance_api.get_symbol_info(symbol))

ETH: 1.0723
None
BTC: 1.0
None
LTC: 4.0
None
BNB: 1.0
None
USDT: 13233.848044
None
TRX: 1478.0
None
XRP: 170.0
None
NEO: 67.05
None
QTUM: 226.0
None
GAS: 166.0
None
LRC: 5634.0
None
ZRX: 2034.0
None
KNC: 1481.0
None
IOTA: 2888.0
None
LINK: 20.86
None
XVG: 18446.0
None
MTL: 772.0
None
ETC: 24.45
None
ZEC: 5.0
None
DASH: 19.0
None
REQ: 3963.0
None
POWR: 3433.0
None
ENJ: 7888.0
None
STORJ: 2136.0
None
BAT: 3430.0
None
LSK: 1656.0
None
MANA: 1657.0
None
ADX: 4834.0
None
ADA: 600.0
None
XLM: 1316.0
None
ICX: 4291.0
None
RLC: 471.0
None
PIVX: 3918.0
None
STEEM: 4134.0
None
ZIL: 18446.0
None
ONT: 4044.0
None
WAN: 5059.0
None
SYS: 14066.0
None
TUSD: 10000.0
None
ZEN: 56.0
None
THETA: 695.0
None
IOTX: 18446.0
None
DATA: 18446.0
None
SC: 18446.0
None
DENT: 18446.0
None
ARDR: 6267.0
None
HOT: 18446.0
None
VET: 18446.0
None
RVN: 18446.0
None
USDC: 10000.0
None
ONG: 3271.0
None
FET: 883.0
None
IOST: 18446.0
None
CELR: 18446.0
None
ATOM: 84.3
None
PHB: 972.0
None
TFUEL: 15910.0
None
ONE: 18446.0
Non

In [4]:

# Throw if position change is older than 2 days; things might not have been updated and we don't
# hold to trade outdated positions.
# TODO: Fix amount of days
if (future_weights_date < date.today() - timedelta(days=10)):
	raise Exception(f'Future position sizes are too old: From {future_weights_date} while they should not be older than 2 days')

buying_power = sum([
	p[1] for p in current_positions.items()
])
print(f'Current buying power is {buying_power}')

Current buying power is 2826276.191944


In [5]:
# Get current prices of future instruments
current_prices = {}

# Go through symbols 1 by 1: If we fail, we want to know where
for symbol in future_weights.index:
    try:
        ticker = f'{symbol}USDT'
        price = binance_api.get_price([ticker])
        print(f'Price for {symbol} is {price}')
        current_prices[symbol] = price
    except Exception as e:
        print(f'Error fetching {symbol}: {e}')
        # Make sure we fail here: 
        sys.exit(1)
print(f'Prices are {current_prices}')

future_positions = dict([
	(name, weight * buying_power / current_prices[name])
	for (name, weight) in future_weights.items()
])
print(f'Future positions are {future_positions}')


Price for AVAX is 22.63
Price for BTC is 112794.01
Price for CELO is 0.3128
Price for CRV is 0.5889
Price for DASH is 45.68
Price for LTC is 98.17
Price for MANA is 0.2713
Price for NEAR is 2.454
Price for SNX is 1.977
Price for ZEC is 237.21
Prices are {'AVAX': 22.63, 'BTC': 112794.01, 'CELO': 0.3128, 'CRV': 0.5889, 'DASH': 45.68, 'LTC': 98.17, 'MANA': 0.2713, 'NEAR': 2.454, 'SNX': 1.977, 'ZEC': 237.21}
Future positions are {'AVAX': 18461.339182596163, 'BTC': 6.366233911049593, 'CELO': 866256.7686636861, 'CRV': 297653.0043275102, 'DASH': 1975.4285782684506, 'LTC': 3801.6053087038385, 'MANA': 885011.582920204, 'NEAR': 53970.2597432928, 'SNX': 103713.39086983429, 'ZEC': 856.3756128046706}


In [8]:
orders = position_size.get_order_sizes(current_positions, future_positions)
# Do negative orders (sell / reduce) first; we don't go short, so that's fine
sorted_orders = dict(sorted(orders.items(), key=lambda x: x[1]))

print('Sorted orders are:')
print(sorted_orders)

for (name, size) in sorted_orders.items():
	side = 'BUY' if size > 0 else 'SELL'
	symbol = f'{name}USDT'
	# symbol = f'{name}'
	print(f'Order {side} for {abs(size)} of {symbol}')
	order = binance_api.place_spot_order(
		symbol = symbol,
		side = side,
		order_type = 'MARKET',
		quote_quantity = abs(size)  # Buy $x worth
	)
	time.sleep(4)
	print(f'Order done: {order}')

Sorted orders are:
{'TRY': -18466.0, 'PLN': -18466.0, 'UAH': -18466.0, 'ARS': -18466.0, 'COP': -18466.0, 'CZK': -18466.0, 'MXN': -18466.0, 'JPY': -18466.0, 'ZAR': -18466.0, 'RON': -18466.0, 'MEME': -18446.0, 'ANIME': -18446.0, 'REZ': -18446.0, 'QUICK': -18446.0, 'FLOKI': -18446.0, 'ROSE': -18446.0, 'ONE': -18446.0, 'GALA': -18446.0, 'HOT': -18446.0, 'GUN': -18446.0, 'ASTR': -18446.0, 'JASMY': -18446.0, 'FLM': -18446.0, 'ARPA': -18446.0, 'DOGS': -18446.0, 'BEAMX': -18446.0, 'MDT': -18446.0, 'NKN': -18446.0, 'SKL': -18446.0, 'ACA': -18446.0, 'LINEA': -18446.0, 'SUN': -18446.0, 'G': -18446.0, 'QI': -18446.0, 'XEC': -18446.0, 'A2Z': -18446.0, 'XVG': -18446.0, 'GPS': -18446.0, 'RDNT': -18446.0, 'HMSTR': -18446.0, 'COS': -18446.0, 'ZIL': -18446.0, 'SLP': -18446.0, 'QKC': -18446.0, '1000SATS': -18446.0, 'MBL': -18446.0, 'PEOPLE': -18446.0, 'DGB': -18446.0, '1000CAT': -18446.0, 'SHIB': -18446.0, 'FUN': -18446.0, 'BOME': -18446.0, 'USTC': -18446.0, 'UTK': -18446.0, 'TLM': -18446.0, 'AMP': -1844

KeyboardInterrupt: 